## Table Extraction Performance Metrics

In [1]:
import pandas as pd
from Levenshtein import distance as lev_dist

***Basic Dimensions***

Metric: Overlap

Formula:
$$
\frac{P(T_1 \cap T_2)}{P(T_1 \cup T_2)}
$$

Explanation: This metric measures the overlap when the tables are superimposed on one another by looking at the ratio of shared cells to total cells (i.e. intersection vs union). This then equally treats small tables that are off by small amounts and larger tables that are incorrect by proportionally similar counts. This simply measures dimension similarity between tables and does not look at their actual contents, just whether cells exist or not. This measure is symmetrical and ressembles the Jaccard similarity index.

Scale:  $0 \leq x \leq 1$. It is similar to a similarity percentage in that a case with two tables of identical dimensions will return a value of 1 (100%). The closer it is to 1, the more alike the dimensions of the two tables are, while a value near 0 indicates very dissimilar dimensions. No table pair will ever return a value of exactly 0 since all tables will have at least one cell and thus be shared between the intersection of the two (unless no table is read).

In [2]:
def overlap(df1, df2):
    # Find intersection area (mulitplication of dim mins)
    xmin = min(df1.shape[0], df2.shape[0])
    ymin = min(df1.shape[1], df2.shape[1])
    intersection = xmin*ymin
    # Find union area (multiplication of dim maxs minus multiplication of dim max-min diffs)
    xmax = max(df1.shape[0], df2.shape[0])
    ymax = max(df1.shape[1], df2.shape[1])
    union = xmax*ymax - (xmax-xmin)*(ymax-ymin)
    # Calc
    overlap_pct = intersection / union
    return round(overlap_pct, 3)

***Contents***

Metric: String Similarity

Formula:
$$
1 - \frac{Lev(~ str(T_1) ~ , ~ str(T_2) ~)}{max(~len(str(T_1))~,~len(str(T_2))~)}
$$

Explanation: First, the tables are transformed into string values which are then compared by calculating the Levenshtein distance which measures the number of edits required to obtain one string when starting from the other. To account for larger strings requiring more edits, this is then divided by the length of the bigger of the two strings, giving us edits per character. The scale is then inverted by doing 1 minus this ratio. This measure is symmetrical.

Scale:  $0 \leq x \leq 1$. The fraction on its own is similar to a distance metric where identical elements will return a value of zero while a value of 1 represents completely different variables. However, this measure takes its complement, meaning 1 minus the fraction/percentage (1-%). This is done to align the metric with all the other table extraction performance statistics which act as similarity percentages. Therefore, the closer it is to 1, the more alike the tables contents are, while a value near 0 indicates very dissimilar table strings.

In [3]:
def string_similarity(df1, df2):
    # Transform tables to strings
    str1 = df1.to_string()
    str2 = df2.to_string()
    # Calc
    edits_per_char = lev_dist(str1, str2) / max(len(str1), len(str2))
    return round(1-edits_per_char, 3)

***Structure***

Metric: Completeness

Formula:
$$
\frac{\text{\# of completely identified elements}}{\text{\# of real elements}}
$$

Explanation: When an element gets split into multiple parts, the detected output elements are classified as incomplete. As a result, a completely identified column for example must contain all of its cells in the processed output table. So it essentially looks to capture missed and split table elements.

Scale:  $0 \leq x \leq 1$. It should be interpreted as a percentile score where a score of 1 (or 100%) indicates a perfectly complete processed table that in each cell, contains ALL the items of the original table's respective cells. Meanwhile, a lower score near zero indicates the opposite.

In [4]:
def completeness(df1, df2):
    # Transform tables to lists of lists
    vals1 = df1.values.tolist()
    vals2 = df2.values.tolist()
    # Initialize numerator & denominator counters
    compl_count = 0
    real_total = df1.size
    # Loop through values
    for i in range(0, len(vals1)):
        row = vals1[i]
        for j in range(0, len(row)):
            cell1 = row[j]
            # Verify if cell1 is contained in cell2 (completely identified)
            try:
                cell2 = vals2[i][j]
                if cell1 in cell2:
                    compl_count += 1
            except IndexError:
                pass
    # Calc
    completeness_score = compl_count / real_total
    return round(completeness_score, 3)

Metric: Purity

Formula:
$$
\frac{\text{\# of purely detected elements}}{\text{\# of detected elements}}
$$

Explanation: When parts of multiple elements from the true table get merged into one, the detected output elements is classified as impure. Thus, a pure element is one whose components belong to only one original element. So it essentially looks to capture improperly metrged and false table elements.

Scale:  $0 \leq x \leq 1$. It should be interpreted as a percentile score where a score of 1 (or 100%) indicates a perfectly pure processed table that in each cell, contains ONLY the items of the original table's respective cells. Meanwhile, a lower score near zero indicates the opposite.

In [5]:
def purity(df1, df2):
    # Transform tables to lists of lists
    vals1 = df1.values.tolist()
    vals2 = df2.values.tolist()
    # Initialize numerator & denominator counters
    pure_count = 0
    detected_total = df2.size
    # Loop through values
    for i in range(0, len(vals2)):
        row = vals2[i]
        for j in range(0, len(row)):
            cell2 = row[j]
            # Verify if cell2 is contained in cell1 (purely detected)
            try:
                cell1 = vals1[i][j]
                if cell2 in cell1:
                    pure_count += 1
            except IndexError:
                pass
    # Calc
    purity_score = pure_count / detected_total
    return round(purity_score, 3)


***Location***

Metric: Precision

Formula:
$$
\frac{\text{\# of correct proto-links}}{\text{\# of detected proto-links}}
$$

Explanation: A proto-link (PL) is defined as a connection between two neighbouring cells within a table. This metric looks to quantify proper cell location by measuring cell pairs that continue to share a border through the processed table output. It is similar to the recall metric but the ratio is over a different numerator, the total detected proto-links rather than total real proto-links. It is essentially calculated as the mirrored version of the recall metric.

Scale:  $0 \leq x \leq 1$. This metric behaves like a similarity percentage with a score of 1 (100%) indicating completely identical tables while a score of 0 demonstrates entirely dissimilar tables.

In [6]:
def precision(df1, df2):
    # Transform tables to lists of lists
    vals1 = df1.values.tolist()
    vals2 = df2.values.tolist()
    r,c = df2.shape
    # Initialize numerator & denominator counters
    correct_pl = 0
    detected_pl = 2*r*c - r - c
    # Loop through connections within rows
    for i in range(0, len(vals2)):
        row = vals2[i]
        for j in range(0, len(row)-1):
            cells2 = row[j:j+2]
            # Verify if pair of cells2 is equal to pair of cells1 (correct PL)
            try:
                cells1 = vals1[i][j:j+2]
                if cells2 == cells1:
                    correct_pl += 1
            except IndexError:
                pass
    # Loop through connections within columns
    tvals1 = df1.transpose().values.tolist()
    tvals2 = df2.transpose().values.tolist()
    for i in range(0, len(tvals2)):
        col = tvals2[i]
        for j in range(0, len(col)-1):
            cells2 = col[j:j+2]
            # Verify if pair of cells2 is equal to pair of cells1 (correct PL)
            try:
                cells1 = tvals1[i][j:j+2]
                if cells2 == cells1:
                    correct_pl += 1
            except IndexError:
                pass
    # Calc
    precision_score = correct_pl / detected_pl
    return round(precision_score, 3)

Metric: Recall

Formula:
$$
\frac{\text{\# of correct proto-links}}{\text{\# of total proto-links}}
$$

Explanation: A proto-link (PL) is defined as a connection between two neighbouring cells within a table. This metric looks to quantify proper cell location by measuring cell pairs that continue to share a border through the processed table output. It is similar to the precision metric but the ratio is over a different numerator, the total real proto-links rather than total detected proto-links. It is essentially calculated as the mirrored version of the precision metric.

Scale:  $0 \leq x \leq 1$. This metric behaves like a similarity percentage with a score of 1 (100%) indicating completely identical tables while a score of 0 demonstrates entirely dissimilar tables.

In [7]:
def recall(df1, df2):
    # Transform tables to lists of lists
    vals1 = df1.values.tolist()
    vals2 = df2.values.tolist()
    r,c = df1.shape
    # Initialize numerator & denominator counters
    correct_pl = 0
    total_pl = 2*r*c - r - c
    # Loop through connections within rows
    for i in range(0, len(vals1)):
        row = vals1[i]
        for j in range(0, len(row)-1):
            cells1 = row[j:j+2]
            # Verify if pair of cells1 is equal to pair of cells2 (correct PL)
            try:
                cells2 = vals2[i][j:j+2]
                if cells1 == cells2:
                    correct_pl += 1
            except IndexError:
                pass
    # Loop through connections within columns
    tvals1 = df1.transpose().values.tolist()
    tvals2 = df2.transpose().values.tolist()
    for i in range(0, len(tvals1)):
        col = tvals1[i]
        for j in range(0, len(col)-1):
            cells1 = col[j:j+2]
            # Verify if pair of cells1 is equal to pair of cells2 (correct PL)
            try:
                cells2 = tvals2[i][j:j+2]
                if cells1 == cells2:
                    correct_pl += 1
            except IndexError:
                pass
    # Calc
    recall_score = correct_pl / total_pl
    return round(recall_score, 3)

#### Test Extraction Performance

Run all performance metrics for a table and its processed output

In [8]:
def all_metrics(df1, df2):
    # Clean & format dataframes
    df1 = df1.fillna('').astype(str)
    df2 = df2.fillna('').astype(str)
    # Run all metric functions
    results = {'Overlap':            overlap(df1, df2),
               'String Similarity':  string_similarity(df1, df2), 
               'Completeness':       completeness(df1, df2),
               'Purity':             purity(df1, df2),
               'Precision':          precision(df1, df2),
               'Recall':             recall(df1, df2)}
    return results

Calculate all metrics for all table pairs

In [9]:
def test_tables(tables):
    # Initialize list to store metrics/results
    results_lst = []
    # Run all metrics for each table
    for t in tables:
        df1, df2 = tables[t]
        results_lst.append(all_metrics(df1, df2))
    # Return dataframe with all metrics (columns) for each table (rows)
    return pd.DataFrame(results_lst, index=tables.keys())